In [ ]:
from onediff.infer_compiler import oneflow_compile
from onediff.schedulers import EulerDiscreteScheduler
from diffusers import StableDiffusionPipeline
import oneflow as flow
import torch

In [ ]:
model_id = "runwayml/stable-diffusion-v1-5"
prompt = "photo of a cat"
height = 512
width = 512
steps = 20

In [ ]:
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    scheduler=scheduler,
    torch_dtype=torch.float16,
    safety_checker=None,
)
pipe = pipe.to("cuda")
pipe.unet = oneflow_compile(
    pipe.unet,
    options={
        "graph_file": "/stable-diffusion-v1-5_graph",
        "graph_file_device": "cuda",
    }
)

In [ ]:
images = pipe(
    prompt, height=height, width=width, num_inference_steps=steps
).images
image = images[0]
image